## Đồ Án Cuối Kì Nhập Môn Khoa Học Dữ Liệu

### Đề tài: Dự báo lượng mưa từ những thông số thời tiết khác như nhiệt độ, độ ẩm, áp suất không khí, ...

Thông tin nhóm
1. Nguyễn Văn Hậu - 18120359
2. Nguyễn Tấn Thìn - 18120085

## 1. Thu thập data

#### Tổng quan về Visual Crossing Timeline Weather API
- Website: `https://www.visualcrossing.com/weather-api`
- Documents: `https://www.visualcrossing.com/resources/documentation/weather-api/weather-api-documentation/#timeline`
- API cung cấp thông tin về thời tiết trong quá khứ, giữa 2 khoảng thời gian cho trước tại một địa điểm xác định và có thể có thêm thông tin dự báo thời tiết (nhưng ta sẽ không lấy thông tin này)
- Cần tạo một tài khoản để được cung cấp API key
- Mỗi API key miễn phí bị giới hạn chỉ được thu thập tối đa 1000 kết quả trả về trong một ngày

#### Cú pháp API
`https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/[location]/[date1]/[date2]?key=YOUR_API_KEY`
với:
- `location` là địa điểm cần lấy thông tin thời tiết
- `date1` là thời gian bắt đầu
- `date2` là thời gian kết thúc
- `YOUR_API_KEY` là api key cung cấp cho mỗi tài khoản đăng kí

#### Cách thu thập data
- Chúng ta tiến hành thu thập dữ liệu thời tiết trong quá khứ theo từng ngày trong thời gian từ năm 2017 đến năm 2020, cụ thể là từ **1/1/2017** đến **31/12/2020**
- Vì giới hạn mỗi API key chỉ được thu thập **tối đa 1000** kết quả một ngày nên ta sẽ dùng 2 API key, mỗi API key tương ứng thu thập dữ liệu năm 2017-2018 và năm 2019-2020

### Import

In [178]:
%matplotlib inline
import requests
import json
import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

### Chuẩn bị các hàm cần thiết

Hàm thu thập data giữa 2 ngày

In [2]:
def collect_data(start_date, end_date, api_key):
    url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Ho%20Chi%20Minh/{start_date}/{end_date}?unitGroup=metric&key={api_key}&include=obs'
    weather_data = []
    success = False
    while success == False:
        print(f"GET: {url}")
        r = requests.get(url)
        if r.ok == True:
            # lấy kết quả json
            pydata = json.loads(r.text)
            weather_data.extend(pydata["days"])
            print("Successful! Remaining cost:", pydata['remainingCost'])
            print()

            success = True
        else:
            print("Fail! Try again. Remaining cost:", pydata['remainingCost'])
            time.sleep(5)
        
    return weather_data

Hàm thu thập toàn bộ data

In [3]:
def collect_all_data(start_year, end_year, api_key):
    years = list(range(start_year, end_year + 1))
    print('List years:', years)
    month_days = [
        ((1, 1), (3, 31)), 
        ((4, 1), (6, 30)),
        ((7, 1), (9, 30)),
        ((10, 1), (12, 31))
    ]
    all_weather_data = []
    
    for year in years:
        for month_day in month_days:
            start_date = f'{year}-{month_day[0][0]}-{month_day[0][1]}'
            end_date = f'{year}-{month_day[1][0]}-{month_day[1][1]}'
            print(f'({start_date} -> {end_date})', end='\n')
#             print(url)
            weather_data = collect_data(start_date, end_date, api_key)
            all_weather_data.extend(weather_data)
        print()
        
    return all_weather_data

### Tiến hành thu thập data

In [4]:
api_key1 = 'A29UZSNHSG8P8EPR6A4CM2UC6'
api_key2 = 'YXRHHRS6QX82D58S867TW6YBF'
all_weather_data = []

# Thu thập lần 1
start_year = 2017
end_year = 2018
weather_data = collect_all_data(start_year, end_year, api_key1)
all_weather_data.extend(weather_data)

# Thu thập lần 2
start_year = 2019
end_year = 2020
weather_data = collect_all_data(start_year, end_year, api_key2)
all_weather_data.extend(weather_data)

print('Number of data:', len(all_weather_data))

List years: [2017, 2018]
(2017-1-1 -> 2017-3-31)
GET: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Ho%20Chi%20Minh/2017-1-1/2017-3-31?unitGroup=metric&key=A29UZSNHSG8P8EPR6A4CM2UC6&include=obs
Successful! Remaining cost: 910

(2017-4-1 -> 2017-6-30)
GET: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Ho%20Chi%20Minh/2017-4-1/2017-6-30?unitGroup=metric&key=A29UZSNHSG8P8EPR6A4CM2UC6&include=obs
Successful! Remaining cost: 819

(2017-7-1 -> 2017-9-30)
GET: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Ho%20Chi%20Minh/2017-7-1/2017-9-30?unitGroup=metric&key=A29UZSNHSG8P8EPR6A4CM2UC6&include=obs
Successful! Remaining cost: 727

(2017-10-1 -> 2017-12-31)
GET: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Ho%20Chi%20Minh/2017-10-1/2017-12-31?unitGroup=metric&key=A29UZSNHSG8P8EPR6A4CM2UC6&include=obs
Successful! Remaining cost: 635


(2018-1-1 

### Lưu data xuống file csv

Tạo dataframe và lưu dữ liệu xuống file `historical_weather_data.csv`

In [106]:
data_df = pd.DataFrame(all_weather_data)
data_df.rename(lambda name: name[0].upper() + name[1:], axis='columns', inplace=True)
data_df.to_csv('historical_weather_data.csv', index=False)
data_df.head()

,Datetime,DatetimeEpoch,Tempmax,Tempmin,Temp,Feelslikemax,Feelslikemin,Feelslike,Dew,Humidity,...,Solarenergy,Sunrise,SunriseEpoch,Sunset,SunsetEpoch,Moonphase,Conditions,Icon,Stations,Source
0,2017-01-01,1483203600,34.0,23.0,27.9,35.7,23.0,29.4,22.2,74.98,...,None,06:11:33,1483225893,17:41:54,1483267314,0.05,"Rain, Partially cloudy",cloudy,"[48894099999, 48900099999, VVTS]",obs
1,2017-01-02,1483290000,33.0,23.0,28.4,36.1,23.0,30.1,21.3,67.16,...,None,06:11:57,1483312317,17:42:27,1483353747,0.09,Partially cloudy,partly-cloudy-day,"[48894099999, 48900099999, VVTS]",obs
2,2017-01-03,1483376400,33.8,24.1,28.2,35.7,24.1,29.8,21.7,69.77,...,None,06:12:20,1483398740,17:43:00,1483440180,0.13,Partially cloudy,partly-cloudy-day,"[48894099999, 48900099999, VVTS]",obs
3,2017-01-04,1483462800,32.8,24.0,27.8,35.2,24.0,29.5,21.6,70.70,...,None,06:12:42,1483485162,17:43:33,1483526613,0.18,Partially cloudy,partly-cloudy-day,"[48894099999, 48900099999, VVTS]",obs
4,2017-01-05,1483549200,32.0,25.0,28.5,34.9,25.0,30.4,22.3,71.11,...,None,06:13:03,1483571583,17:44:05,1483613045,0.23,Partially cloudy,partly-cloudy-day,"[48894099999, 48900099999, VVTS]",obs


## 2. Khám phá dữ liệu (để đưa ra câu hỏi)

Dữ liệu thời tiết thu thập được ở bước trước là dữ liệu tại vị trí quận 1 của Thành phố Hồ Chí Minh.  

Load dữ liệu từ file `historical_weather_data.csv`

In [263]:
data_df = pd.read_csv('historical_weather_data.csv')
data_df.head()

,Datetime,DatetimeEpoch,Tempmax,Tempmin,Temp,Feelslikemax,Feelslikemin,Feelslike,Dew,Humidity,...,Solarenergy,Sunrise,SunriseEpoch,Sunset,SunsetEpoch,Moonphase,Conditions,Icon,Stations,Source
0,2017-01-01,1483203600,34.0,23.0,27.9,35.7,23.0,29.4,22.2,74.98,...,NaN,06:11:33,1483225893,17:41:54,1483267314,0.05,"Rain, Partially cloudy",cloudy,"['48894099999', '48900099999', 'VVTS']",obs
1,2017-01-02,1483290000,33.0,23.0,28.4,36.1,23.0,30.1,21.3,67.16,...,NaN,06:11:57,1483312317,17:42:27,1483353747,0.09,Partially cloudy,partly-cloudy-day,"['48894099999', '48900099999', 'VVTS']",obs
2,2017-01-03,1483376400,33.8,24.1,28.2,35.7,24.1,29.8,21.7,69.77,...,NaN,06:12:20,1483398740,17:43:00,1483440180,0.13,Partially cloudy,partly-cloudy-day,"['48894099999', '48900099999', 'VVTS']",obs
3,2017-01-04,1483462800,32.8,24.0,27.8,35.2,24.0,29.5,21.6,70.70,...,NaN,06:12:42,1483485162,17:43:33,1483526613,0.18,Partially cloudy,partly-cloudy-day,"['48894099999', '48900099999', 'VVTS']",obs
4,2017-01-05,1483549200,32.0,25.0,28.5,34.9,25.0,30.4,22.3,71.11,...,NaN,06:13:03,1483571583,17:44:05,1483613045,0.23,Partially cloudy,partly-cloudy-day,"['48894099999', '48900099999', 'VVTS']",obs


### Dữ liệu có bao nhiêu dòng, bao nhiêu cột

In [109]:
data_df.shape

(1461, 32)

### Ý nghĩa của mỗi dòng

Mỗi dòng là thông tin thời tiết ghi nhận được của một ngày trong năm

### Ý nghĩa của mỗi cột

Dữ liệu có rất nhiều cột nhưng chúng ta chỉ quan tâm một số cột quan trọng:
- `Datetime`: ngày ghi nhận thông tin (yyyy-m-d)
- `Tempmax`: nhiệt độ cao nhất trong ngày (Celsius)
- `Tempmin`: nhiệt độ thấp nhất trong ngày (Celsius)
- `Temp`: nhiệt độ trung bình trong ngày (Celsius)
- `Dew`: điểm sương, là nhiệt độ mà tại đó hơi nước trong khối không khí ngưng đọng thành nước lỏng, độ ẩm tương đối của khối không khí đạt 100% (Celsius)
- `Humidity`: độ ẩm tương đối (%)
- `Precip`: lượng mưa (mm)
- `Precipcover`: tỉ lệ thời gian mưa trong ngày (%)
- `Windgust`: tốc độ cơn gió mạnh (tốc độ tối đa của gió thổi trong khoảng thời gian ngắn và phải trên một mức nào đó mới được ghi nhận) (kph)
- `Windspeed`: tốc độ gió trung bình (kph)
- `Winddir`: hướng gió đo so với hướng Bắc (degrees)
- `Pressure`: áp suất không khí (millibars)
- `Cloudcover`: tỉ lệ mây bao phủ bầu trời (%)
- `Visibility`: tầm nhìn xa ban ngày (km)
- `Conditions`: hiện tượng thời tiết ghi nhận được như sấm sét, mưa, ...

Chi tiết cụ thể hơn về thông tin các cột có tại `https://www.visualcrossing.com/resources/documentation/weather-data/weather-data-documentation/`

### Kiểm tra các dòng có bị lặp không

In [110]:
data_df.index.duplicated().sum()

0

### Đặt câu hỏi

**Câu hỏi:** *Output* - lượng mưa (Precip) được tính từ *input* - các thông số khác (nhiệt độ, độ ẩm, tốc độ gió, áp suất, ...) như thế nào?  
  
**Ý nghĩa:** Khi có thể dự đoán được lượng mưa thì sẽ giúp ích cho việc lập kế hoạch, tổ chức sự kiện, hoặc để tối ưu hiệu suất trong nông nghiệp và nhiều lĩnh vực khác.

## 3. Tách tập, tiền xử lí và khám phá dữ liệu

### Khám phá dữ liệu cột output (để tách tập)

Kiểu dữ liệu của cột output

In [111]:
data_df['Precip'].dtype

dtype('float64')

Kiếm tra cột output có giá trị thiếu không

In [112]:
data_df['Precip'].isna().sum()

0

Phân bố của cột output

In [113]:
data_df['Precip'].describe()

count    1461.000000
mean        4.154004
std        13.286562
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max       227.200000
Name: Precip, dtype: float64

Nhận xét: Giá trị ở cột output phần lớn là nhỏ, tại TP HCM quanh năm thì lượng mưa cũng không cao như các tỉnh khác (ví dụ các tỉnh miền Trung)

In [114]:
(data_df['Precip'] > 4.15).sum()

269

### Tiền xử lí (tách các tập)

Tiến hành tách tập data thu được thành 3 tập huấn luyện, validation và tập kiểm tra

In [264]:
# Tách X và y
y_sr = data_df['Precip']
X_df = data_df.drop(columns=['Precip', 'Precipprob', 'Precipcover'])

In [265]:
# Tách tập kiểm tra với tỉ lệ 30%
train_valid_X_df, test_X_df, train_valid_y_sr, test_y_sr = train_test_split(
    X_df, y_sr, test_size = 0.3, random_state=0
)

In [266]:
# Tách tập validation từ tập còn lại với tỉ lệ 30%
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(
    train_valid_X_df, train_valid_y_sr, test_size = 0.3, random_state=0
)

Tập kiểm tra

In [118]:
test_X_df.shape

(439, 29)

In [119]:
test_y_sr.shape

(439,)

Tập validation

In [120]:
val_X_df.shape

(307, 29)

In [121]:
val_y_sr.shape

(307,)

Tập huấn luyện

In [122]:
train_X_df.shape

(715, 29)

In [123]:
train_y_sr.shape

(715,)

### Khám phá dữ liệu (tập huấn luyện)

#### Kiểu dữ liệu hiện tại của mỗi cột

In [124]:
train_X_df.dtypes

Datetime           object
DatetimeEpoch       int64
Tempmax           float64
Tempmin           float64
Temp              float64
Feelslikemax      float64
Feelslikemin      float64
Feelslike         float64
Dew               float64
Humidity          float64
Snow              float64
Snowdepth         float64
Windgust          float64
Windspeed         float64
Winddir           float64
Pressure          float64
Cloudcover        float64
Visibility        float64
Solarradiation    float64
Solarenergy       float64
Sunrise            object
SunriseEpoch        int64
Sunset             object
SunsetEpoch         int64
Moonphase         float64
Conditions         object
Icon               object
Stations           object
Source             object
dtype: object

Nhận xét: Phần lớn cột trong dữ liệu input là ở dạng số

#### Phân bố của các cột dữ liệu dạng số

In [125]:
num_cols = list(train_X_df.dtypes[train_X_df.dtypes == float].index)
num_cols

['Tempmax',
 'Tempmin',
 'Temp',
 'Feelslikemax',
 'Feelslikemin',
 'Feelslike',
 'Dew',
 'Humidity',
 'Snow',
 'Snowdepth',
 'Windgust',
 'Windspeed',
 'Winddir',
 'Pressure',
 'Cloudcover',
 'Visibility',
 'Solarradiation',
 'Solarenergy',
 'Moonphase']

In [126]:
df = train_X_df[num_cols]

def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)

df.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

,Tempmax,Tempmin,Temp,Feelslikemax,Feelslikemin,Feelslike,Dew,Humidity,Snow,Snowdepth,Windgust,Windspeed,Winddir,Pressure,Cloudcover,Visibility,Solarradiation,Solarenergy,Moonphase
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.00,100.0,100.0,83.6,0.0,0.0,1.0,0.0,0.0,100.0,100.0,0.0
min,24.6,20.0,22.8,0.0,0.0,22.8,15.2,50.17,NaN,NaN,25.9,8.1,54.8,1004.4,22.5,6.2,NaN,NaN,0.0
lower_quartile,32.0,24.0,27.4,36.2,24.0,29.5,22.5,71.30,NaN,NaN,33.5,15.2,143.2,1008.5,44.4,9.0,NaN,NaN,0.2
median,33.0,25.0,28.3,38.6,25.0,31.2,24.4,79.30,NaN,NaN,40.7,18.4,182.1,1009.8,52.6,9.6,NaN,NaN,0.5
upper_quartile,34.0,26.0,29.0,40.6,26.0,33.0,25.2,85.40,NaN,NaN,48.2,21.3,236.8,1011.3,60.0,10.1,NaN,NaN,0.8
max,37.0,29.0,32.1,48.5,35.6,39.3,26.9,97.77,NaN,NaN,74.2,40.7,316.2,1015.7,99.6,10.7,NaN,NaN,1.0


#### Phân bố của các cột dữ liệu không phải dạng số

In [127]:
pd.set_option('display.max_colwidth', 200) # Để nhìn rõ hơn
cat_cols = ['Datetime', 'Sunrise', 'Sunset', 'Conditions', 'Icon']
cat_multi_cols = ['Conditions']
df = train_X_df[cat_cols]

def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    if c.name in cat_multi_cols:
        s = c.str.split(', ').explode().value_counts(normalize=True)
        return dict((s * 100).round(1))
    else:
        return dict((c.value_counts(normalize=True) * 100).round(1))
    
df.agg([missing_ratio, num_values, value_ratios])

,Datetime,Sunrise,Sunset,Conditions,Icon
missing_ratio,0,0,0,0,0
num_values,715,579,589,5,5
value_ratios,"{'2019-11-26': 0.1, '2017-02-07': 0.1, '2020-07-02': 0.1, '2020-06-03': 0.1, '2017-11-18': 0.1, '2020-04-23': 0.1, '2020-02-13': 0.1, '2017-07-29': 0.1, '2018-09-12': 0.1, '2018-09-26': 0.1, '2020...","{'05:41:51': 1.1, '05:41:50': 1.0, '05:43:32': 1.0, '05:43:21': 0.7, '05:43:24': 0.6, '05:43:33': 0.6, '05:43:34': 0.6, '06:16:54': 0.6, '05:43:26': 0.6, '05:42:11': 0.4, '05:43:14': 0.4, '05:41:4...","{'18:04:04': 3.6, '18:04:03': 1.5, '18:04:05': 1.3, '18:20:02': 0.7, '17:26:37': 0.7, '17:26:38': 0.6, '18:03:59': 0.6, '18:20:00': 0.6, '18:20:04': 0.6, '18:19:35': 0.4, '18:04:24': 0.4, '17:26:4...","{'Partially cloudy': 68.0, 'Rain': 27.3, 'Overcast': 4.1, 'Clear': 0.6}","{'partly-cloudy-day': 59.9, 'rain': 26.2, 'cloudy': 11.5, 'wind': 1.7, 'clear-day': 0.8}"


In [128]:
data_df['Conditions'].str.split(', ').explode().value_counts(normalize=True)

Partially cloudy    0.666337
Rain                0.276733
Overcast            0.047030
Clear               0.009901
Name: Conditions, dtype: float64

## 4. Tiền xử lí và mô hình hóa dữ liệu

Dữ liệu của ta có khá nhiều cột nhưng nhận thấy chỉ một số cột (có thể) ảnh hưởng đến kết quả lượng mưa. Chẳng hạn:
- Tháng trong năm: tùy vào từng mùa mà có những tháng mưa nhiều và những tháng mưa ít hoặc không mưa
- Ngoài ra còn có nhiệt độ, độ ẩm, tốc độ gió, áp suất, ...

### Tiền xử lí tập huấn luyện

Ta xử lí thêm bớt một số cột như sau:
- Tạo thêm 2 cột là Day và Month lấy thông tin ngày, tháng từ cột Datetime. Sau đó bỏ cột Datetime.
- Bỏ các cột DatetimeEpoch, Feelslikemax, Feelslikemin, Feelslike, Sunrise, SunriseEpoch, Sunset, SunsetEpoch, Moonphase, Icon, Stations, Source vì có lẽ không mang lại nhiều thông tin hữu ích.
- Bỏ các cột Snow, Snowdepth, Windgust, Solarradiation, Solarenergy vì chứa nhiều giá trị trống (NaN).

Tại mỗi dòng của cột Conditions có thể có nhiều giá trị nên ta xử lí cột Conditions như sau:
- Những dòng có xuất hiện trạng thái Rain thì nhận giá trị 1, ngược lại nhận giá trị 0

Với các cột dữ liệu dạng số ta sẽ điền giá trị thiếu bằng mean của cột bằng cách sử dụng `SimpleImputer`

In [154]:
def col_adder_dropper(X_df):
    month_sr = X_df['Datetime'].str.split('-').apply(lambda x: x[1])
    day_sr = X_df['Datetime'].str.split('-').apply(lambda x: x[2])
        
    # Bỏ cột các cột không cần thiết
    del_cols = ['Datetime', 'DatetimeEpoch', 'Feelslikemax', 'Feelslikemin', 'Feelslike', \
                'Sunrise', 'SunriseEpoch', 'Sunset', 'SunsetEpoch', 'Moonphase', 'Icon', \
                'Stations', 'Source', 'Snow', 'Snowdepth', 'Windgust', 'Solarradiation', 'Solarenergy']
    X_output = X_df.drop(columns=del_cols)
    
    # Thêm cột Month, Day
    X_output.insert(0, 'Day', day_sr)
    X_output.insert(0, 'Month', month_sr)
    
    return X_output

In [226]:
# Xử lí cột Conditions, nếu trong các hiện tượng ghi nhận có Rain thì sẽ mang giá trị 1,
# ngược lại thì là 0
def col_condition_trans(X_df):
    condition_sr = train_X_df['Conditions'].str.split(', ').apply(lambda x: 1 if 'Rain' in x else 0)
    X_output = X_df.copy()
    X_output['Conditions'] = condition_sr
    return X_output

In [234]:
df = FunctionTransformer(col_adder_dropper).fit_transform(train_X_df)
df = FunctionTransformer(col_condition_trans).fit_transform(df)
df

,Month,Day,Tempmax,Tempmin,Temp,Dew,Humidity,Windspeed,Winddir,Pressure,Cloudcover,Visibility,Conditions
1216,05,01,36.2,27.0,30.8,21.9,61.39,22.3,132.5,1009.2,34.5,10.4,0
926,07,16,33.1,24.0,28.1,25.3,85.67,23.8,252.4,1006.3,61.7,9.0,1
609,09,02,34.0,25.4,29.4,25.5,80.82,20.5,266.7,1007.9,64.5,9.4,1
183,07,03,33.0,25.0,28.6,24.5,80.32,18.4,233.0,1008.6,53.2,10.2,0
473,04,19,34.0,27.0,29.9,23.4,69.44,18.0,140.4,1009.8,51.8,10.3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,10,27,30.6,25.0,27.5,22.7,75.42,22.7,316.2,1007.1,73.8,9.8,0
101,04,12,35.0,27.9,30.7,26.5,79.96,16.6,175.0,1008.8,46.0,10.3,0
698,11,30,33.0,25.0,28.9,23.7,74.98,12.6,152.2,1012.8,56.4,9.6,0
602,08,26,32.0,25.0,28.2,25.2,84.90,23.4,252.0,1007.5,68.2,10.1,1


In [242]:
nume_cols = ['Month', 'Day', 'Tempmax', 'Tempmin', 'Temp', 'Dew', 'Humidity', 'Windspeed',
             'Winddir', 'Pressure', 'Cloudcover', 'Visibility']
cate_cols = ['Conditions']

col_transform = make_column_transformer(
    (SimpleImputer(strategy='mean'), nume_cols),
    (SimpleImputer(strategy='most_frequent'), cate_cols),
)

preprocess_pipeline = make_pipeline(
    FunctionTransformer(col_adder_dropper),
    FunctionTransformer(col_condition_trans),
    col_transform,
    StandardScaler()
)
preprocess_pipeline

Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function col_adder_dropper at 0x000001BE5CC4F438>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function col_condition_trans at 0x000001BE60311B88>)),
                ('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer-1',
                                                  SimpleImputer(),
                                                  ['Month', 'Day', 'Tempmax',
                                                   'Tempmin', 'Temp', 'Dew',
                                                   'Humidity', 'Windspeed',
                                                   'Winddir', 'Pressure',
                                                   'Cloudcover',
                                                   'Visibility']),
                                                 ('simpleimputer-2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Conditions'])])),
                ('standardscaler', StandardScaler())])

Tạo preprocess_pipeline

In [243]:
preprocessed_train_X = preprocess_pipeline.fit_transform(train_X_df)
pd.DataFrame(preprocessed_train_X)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.439990,-1.640366,1.920717,1.350955,1.962495,-0.806040,-1.755591,0.690568,-1.015370,-0.360764,-1.441164,1.206249,-0.776620
1,0.129924,0.086801,0.119232,-0.614987,-0.112993,0.786389,0.811499,0.984260,1.223408,-1.781403,0.700125,-0.595116,1.287631
2,0.699839,-1.525222,0.642244,0.302453,0.886316,0.880061,0.298715,0.338137,1.490418,-0.997602,0.920551,-0.080441,1.287631
3,0.129924,-1.410077,0.061120,0.040327,0.271356,0.411700,0.245851,-0.073033,0.861170,-0.654690,0.030972,0.948911,-0.776620
4,-0.724947,0.432235,0.642244,1.350955,1.270666,-0.103498,-0.904476,-0.151351,-0.867861,-0.066839,-0.079241,1.077580,-0.776620
...,...,...,...,...,...,...,...,...,...,...,...,...,...
710,0.984796,1.353391,-1.333578,0.040327,-0.574213,-0.431351,-0.272219,0.768886,2.414685,-1.389503,1.652683,0.434235,-0.776620
711,-0.724947,-0.373777,1.223368,1.940738,1.885625,1.348423,0.207788,-0.425464,-0.221808,-0.556714,-0.535840,1.077580,-0.776620
712,1.269753,1.698824,0.061120,0.040327,0.501966,0.037010,-0.318740,-1.208644,-0.647531,1.402788,0.282888,0.176897,-0.776620
713,0.414882,1.238246,-0.520005,0.040327,-0.036123,0.739553,0.730088,0.905942,1.215939,-1.193553,1.211830,0.820242,1.287631


### Tiền xử lí tập validation

In [244]:
preprocessed_val_X = preprocess_pipeline.transform(val_X_df)

### Kết hợp tiền xử lí và mô hình hóa

Các siêu tham số: alpha, hidden_layer_sizes, max_iter

In [245]:
# Tạo full_pipeline
full_pipeline = make_pipeline(
    preprocess_pipeline,
    MLPRegressor(hidden_layer_sizes=(30), activation='tanh', solver='lbfgs', random_state=0, max_iter=7500)
)

# Thử nghiệm với nhiều giá trị khác nhau của siêu tham số
#mlpregressor__alpha, mlpregressor__max_iter, mlpregressor__hidden_layer_sizes
train_errs = []
val_errs = []
alphas = [0.001, 0.01, 0.1, 1.0, 10, 100]
max_iters = [4000, 5000, 6000, 7000, 8000]
best_val_err = float('inf'); best_alpha = None; best_max_iter = None

for alpha in alphas:
    for max_iter in max_iters:
        full_pipeline.set_params(
            mlpregressor__alpha=alpha,
            mlpregressor__max_iter=max_iter
        )
        full_pipeline.fit(train_X_df, train_y_sr)
        train_err = (1 - full_pipeline.score(train_X_df, train_y_sr))*100
        val_err = (1 - full_pipeline.score(val_X_df, val_y_sr))*100
        
        # Lưu lại lịch sử train_err, val_err
        train_errs.append(train_err)
        val_errs.append(val_err)
        
        # Lưu thông tin mô hình tốt nhất
        if val_err < best_val_err:
            best_val_err = val_err
            best_alpha = alpha
            best_max_iter = max_iter
'Finish!'

'Finish!'

In [246]:
full_pipeline

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('functiontransformer-1',
                                  FunctionTransformer(func=<function col_adder_dropper at 0x000001BE5CC4F438>)),
                                 ('functiontransformer-2',
                                  FunctionTransformer(func=<function col_condition_trans at 0x000001BE60311B88>)),
                                 ('columntransformer',
                                  ColumnTransformer(transformers=[('simpleimputer-1',
                                                                   SimpleImputer(),
                                                                   ['Month',
                                                                    'Day',
                                                                    'Tempmax',
                                                                    'Tempmin',
                                                                    'Temp',
                                                                    'Dew',
                                                                    'Humidity',
                                                                    'Windspeed',
                                                                    'Winddir',
                                                                    'Pressure',
                                                                    'Cloudcover',
                                                                    'Visibility']),
                                                                  ('simpleimputer-2',
                                                                   SimpleImputer(strategy='most_frequent'),
                                                                   ['Conditions'])])),
                                 ('standardscaler', StandardScaler())])),
                ('mlpregressor',
                 MLPRegressor(activation='tanh', alpha=1000,
                              hidden_layer_sizes=30, max_iter=8000,
                              random_state=0, solver='lbfgs'))])

In [222]:
full_pipeline.get_params()

{'memory': None,
 'steps': [('pipeline',
   Pipeline(steps=[('functiontransformer-1',
                    FunctionTransformer(func=<function col_adder_dropper at 0x000001BE5CC4F438>)),
                   ('functiontransformer-2',
                    FunctionTransformer(func=<function col_condition at 0x000001BE5FDA1B88>)),
                   ('columntransformer',
                    ColumnTransformer(transformers=[('simpleimputer',
                                                     SimpleImputer(),
                                                     ['Month', 'Day', 'Tempmax',
                                                      'Tempmin', 'Temp', 'Dew',
                                                      'Humidity', 'Windspeed',
                                                      'Winddir', 'Pressure',
                                                      'Cloudcover',
                                                      'Visibility'])])),
                   ('standardscaler'

#### Kết quả tốt nhất

In [247]:
best_alpha

1000

In [248]:
best_max_iter

4000

In [249]:
best_val_err

92.55066329072837

In [250]:
train_errs

[5.199384217995695,
 5.199384217995695,
 5.199384217995695,
 5.199384217995695,
 5.199384217995695,
 5.460345042788184,
 5.460345042788184,
 5.460345042788184,
 5.460345042788184,
 5.460345042788184,
 6.526451970814562,
 6.526451970814562,
 6.526451970814562,
 6.526451970814562,
 6.526451970814562,
 8.187422373785768,
 8.187422373785768,
 8.187422373785768,
 8.187422373785768,
 8.187422373785768,
 49.66604308627562,
 49.66604308627562,
 49.66604308627562,
 49.66604308627562,
 49.66604308627562,
 76.90198103760275,
 76.90198103760275,
 76.90198103760275,
 76.90198103760275,
 76.90198103760275]

In [251]:
val_errs

[149.127261501218,
 149.127261501218,
 149.127261501218,
 149.127261501218,
 149.127261501218,
 167.9083448044429,
 167.9083448044429,
 167.9083448044429,
 167.9083448044429,
 167.9083448044429,
 163.67412563893222,
 163.67412563893222,
 163.67412563893222,
 163.67412563893222,
 163.67412563893222,
 125.49917568087736,
 125.49917568087736,
 125.49917568087736,
 125.49917568087736,
 125.49917568087736,
 97.07524963490053,
 97.07524963490053,
 97.07524963490053,
 97.07524963490053,
 97.07524963490053,
 92.55066329072837,
 92.55066329072837,
 92.55066329072837,
 92.55066329072837,
 92.55066329072837]

#### Huấn luyện lại mô hình với kết quả siêu tham số tốt nhất

In [254]:
full_pipeline.set_params(
            mlpregressor__alpha=best_alpha,
            mlpregressor__max_iter=best_max_iter
        )
full_pipeline.fit(train_X_df, train_y_sr)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('functiontransformer-1',
                                  FunctionTransformer(func=<function col_adder_dropper at 0x000001BE5CC4F438>)),
                                 ('functiontransformer-2',
                                  FunctionTransformer(func=<function col_condition_trans at 0x000001BE60311B88>)),
                                 ('columntransformer',
                                  ColumnTransformer(transformers=[('simpleimputer-1',
                                                                   SimpleImputer(),
                                                                   ['Month',
                                                                    'Day',
                                                                    'Tempmax',
                                                                    'Tempmin',
                                                                    'Temp',
                                                                    'Dew',
                                                                    'Humidity',
                                                                    'Windspeed',
                                                                    'Winddir',
                                                                    'Pressure',
                                                                    'Cloudcover',
                                                                    'Visibility']),
                                                                  ('simpleimputer-2',
                                                                   SimpleImputer(strategy='most_frequent'),
                                                                   ['Conditions'])])),
                                 ('standardscaler', StandardScaler())])),
                ('mlpregressor',
                 MLPRegressor(activation='tanh', alpha=1000,
                              hidden_layer_sizes=30, max_iter=4000,
                              random_state=0, solver='lbfgs'))])

In [255]:
full_pipeline.score(train_X_df, train_y_sr)

0.2309801896239725

### Nháp

In [260]:
# Tính độ đo r^2 trên tập huấn luyện
def compute_mse(y, preds):
    return ((y - preds) ** 2).mean()
def compute_rr(y, preds, baseline_preds):
    return 1 - compute_mse(y, preds) / compute_mse(y, baseline_preds)

In [273]:
full_pipeline = make_pipeline(
    preprocess_pipeline,
    MLPRegressor(hidden_layer_sizes=(30), activation='relu', solver='lbfgs', random_state=0, max_iter=7500)
)

In [274]:
full_pipeline.fit(train_X_df, train_y_sr)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('functiontransformer-1',
                                  FunctionTransformer(func=<function col_adder_dropper at 0x000001BE5CC4F438>)),
                                 ('functiontransformer-2',
                                  FunctionTransformer(func=<function col_condition_trans at 0x000001BE60311B88>)),
                                 ('columntransformer',
                                  ColumnTransformer(transformers=[('simpleimputer-1',
                                                                   SimpleImputer(),
                                                                   ['Month',
                                                                    'Day',
                                                                    'Tempmax',
                                                                    'Tempmin',
                                                                    'Temp',
                                                                    'Dew',
                                                                    'Humidity',
                                                                    'Windspeed',
                                                                    'Winddir',
                                                                    'Pressure',
                                                                    'Cloudcover',
                                                                    'Visibility']),
                                                                  ('simpleimputer-2',
                                                                   SimpleImputer(strategy='most_frequent'),
                                                                   ['Conditions'])])),
                                 ('standardscaler', StandardScaler())])),
                ('mlpregressor',
                 MLPRegressor(hidden_layer_sizes=30, max_iter=7500,
                              random_state=0, solver='lbfgs'))])

Độ chính xác trên tập huấn luyện

In [275]:
compute_rr(train_y_sr, full_pipeline.predict(train_X_df), train_y_sr.mean())

0.942238566562785

In [276]:
full_pipeline.score(train_X_df, train_y_sr)

0.9422385665627846

Độ chính xác trên tập validation

In [277]:
compute_rr(val_y_sr, full_pipeline.predict(val_X_df), train_y_sr.mean())

-0.47328470840232084

In [278]:
full_pipeline.score(val_X_df, val_y_sr)

-0.4734360465984979

In [199]:
full_pipeline.score(train_X_df, train_y_sr)

0.9449171850558643

In [209]:
full_pipeline.score(train_X_df, train_y_sr)

0.9448176494980373